# Description

This notebook intends to gather all the functionalities you'll have to implement for assignment 2.2. You will have to generate an elastic solid, deform it, compute the associated Jacobian of the deformation map $\phi$, and implement pinning constraints. You will also visualize the eigenvectors and eigenvalues of the metric tensor, given a prescribed deformation.

# Load libraries

In [ ]:
import torch
import igl
import meshplot as mp

import sys as _sys
_sys.path.append("../src")

from create_vol_cube import create_vol_cube
from fem_system import *
from eigendecomposition_metric import *

shadingOptions = {
    "flat":True,
    "wireframe":False,   
}

rot = torch.tensor(
    [[1.0,  0.0, 0.0],
     [0.0,  0.0, 1.0],
     [0.0, -1.0, 0.0]]
)

# Load mesh

Several meshes are available for you to play with under `data/`: `ball.obj`, `dinosaur.obj`, and `beam.obj`. You can also uncomment the few commented lines below to manipulate a simple mesh made out of 2 tetrahedra.

In [ ]:
v, _, _, t, _, _ = igl.read_obj("../data/dinosaur.obj")
v = torch.tensor(v)
t = torch.tensor(t)

# v, t = create_vol_cube(10, 3, 3)
# v = torch.tensor(v) * torch.tensor([10.0, 2.0, 2.0]).reshape(1, -1)
# t = torch.tensor(t)

# t = torch.tensor([
#         [0, 1, 2, 3],
#         [1, 2, 3, 4]
#     ])
# v = torch.tensor([
#     [0.0, 0.0, 0.0],
#     [1.0, 0.0, 0.0],
#     [0.0, 1.0, 0.0],
#     [0.0, 0.0, 1.0],
#     [2/3, 2/3, 2/3]
# ])

aabb = torch.max(v, dim=0).values - torch.min(v, dim=0).values
length_scale = torch.mean(aabb)

be = igl.edges(igl.boundary_facets(to_numpy(t)))
e = igl.edges(to_numpy(t))

p = mp.plot(to_numpy(v @ rot.T), to_numpy(t), shading=shadingOptions)

# Manipulate elastic solids

## Instantiation

The rest shape matrices $D_m$ and their inverse matrices $B_m$ are computed during instanciation.

In [ ]:
rho   = 131.0  # [kg.m-3]
solid = FEMSystem(v, t, rho=rho)

## Deform the mesh

Jacobian computation which relies on deformed shape matrices $D_s$.

In [ ]:
v_def = v.clone()
v_def[:, 2] *= 2.0
v_def_pinned = solid.compute_pinned_deformation(v_def)
jac = solid.compute_jacobians(v_def_pinned)
print("Shape of the jacobians: {}".format(list(jac.shape)))

mp.plot(to_numpy(v_def_pinned @ rot.T), to_numpy(t), shading=shadingOptions)

## Visualize some properties of the metric tensor

The metric tensor measures how stretched and sheared directions in the undeformed space are under the deformation $\phi$. It is defined from the Jacobian of the deformation $\mathbf{F}$ as follow (see the handout for a derivation):

$$\mathbf{M} = \mathbf{F}^{\top} \mathbf{F}$$

We intend to plot the eigenvectors coloured by the corresponding eigenvalues in the next cell.

In [ ]:
# We limit ourselves to stretching the mesh in the z direction
# Feel free to experiment with other kinds of deformations!

v_def = v.clone()
v_def[:, 2] *= 2.0
v_def_pinned = solid.compute_pinned_deformation(v_def)
jac = solid.compute_jacobians(v_def_pinned)

square_root_eigvals, eigvecs = compute_eigendecomposition_metric(jac)
plot_eigendecomposition_metric(solid, v_def_pinned, square_root_eigvals, eigvecs, rot, scale=0.1)

## Pin vertices of the mesh

Pass a `pin_idx` to the constructor, compute the mask for deformations.

In [ ]:
maxZ = torch.max(solid.v_rest[:, 2])
pin_idx = list(torch.arange(solid.v_rest.shape[0])[solid.v_rest[:, 2] > maxZ - 0.1 * aabb[2]])

v_def = v.clone()
v_def[:, 2] -= 0.1 * aabb[2]

solid_pinned = FEMSystem(v, t, rho=rho, pin_idx=pin_idx)
v_def_pinned = solid_pinned.compute_pinned_deformation(v_def)

In [ ]:
p = mp.plot(to_numpy(v_def_pinned @ rot.T), to_numpy(t), shading=shadingOptions)
p.add_points(to_numpy(v_def_pinned[pin_idx, :] @ rot.T), shading={"point_color":"black", "point_size": 0.1 * length_scale})